In [7]:
import json
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datetime import datetime

# Load the generator model and tokenizer
generator_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
generator_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

def load_retrieved_data(file_path):
    """Load the retrieved data from a JSON file."""
    context=[]
    with open(file_path, 'r') as f:
        data= json.load(f)
    for i in range(len(data['results'])):
      context.append(data['results'][i]['document'])
    return context

def generate_answer(query, context):
    """Generate an answer based on the given query and context."""
    prompt = f"""You are an expert on the topics related to GPU and specifically CUDA and nvidia. Your task is to analyze the query and then try to use the provided context and your prior knowledge for answering the query. In case of a contradiction between your knowlege and the provided context, consider the context to be more accurate and then try to formulate your answer based on the new information. In say "I don't have enough information to answer that."

Context: {context}

Question: {query}

Answer:"""

    inputs = generator_tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True)
    outputs = generator_model.generate(**inputs, max_length=150, num_return_sequences=1)
    answer = generator_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

def question_answering(query, file_path):
    """Perform question answering using the retrieved data."""
    context = load_retrieved_data(file_path)
    answer = generate_answer(query, context)
    return answer, context

def save_query_results(query, answer, file_path=None):
    """Save the query results to a JSON file."""
    context=load_retrieved_data('/content/1.json')
    retrieved_docs = context
    query_data = {
        "timestamp": datetime.now().isoformat(),
        "query": query,
        "answer": answer,
        "retrieved_documents": retrieved_docs
    }

    if file_path is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        file_path = f"query_results_{timestamp}.json"

    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(query_data, f, ensure_ascii=False, indent=4)

    print(f"Query results saved to {file_path}")

# Main execution
if __name__ == "__main__":
    # Load the retrieved data
    # retrieved_data = load_retrieved_data('/content/1.json')

    # while True:
    #     query = input("Enter your question (or 'quit' to exit): ")
    #     if query.lower() == 'quit':
    #         break

    #     answer, retrieved_docs = question_answering(query, '/content/1.json')

    #     print("\nAnswer:", answer)
    #     print("\nRetrieved Documents:")
    #     for i, doc in enumerate(retrieved_docs, 1):
    #         print(f"{i}. {doc['content'][:100]}...")  # Print first 100 characters of each document

    #     save_query_results(query, answer, retrieved_docs)

    #     print("\n" + "="*50 + "\n")

    query=' How do I install the Toolkit in a different location? ? '
    file_path='/home/ubuntu/project/Steps/retrieved_result/query_results_20240715_211004.json'
    answer,context= question_answering(query,file_path)
    print(answer)

/etc/mod


In [8]:
answer

'/etc/mod'

In [9]:
context

['Frequently Asked Questions 15.1. How do I install the Toolkit in a different location? 15.2. Why do I see “nvcc: No such file or directory” when I try to build a CUDA application? 15.3. Why do I see “error while loading shared libraries: : cannot open shared object file: No such file or directory” when I try to run a CUDA application that uses a CUDA library? 15.4. Why do I see multiple “404 Not Found” errors when updating my repository meta-data on Ubuntu? 15.5. How can I tell X to ignore a GPU for compute-only use? 15.6. Why doesn’t the cuda-repo package install the CUDA Toolkit and Drivers? 15.7. How do I get CUDA to work on a laptop with an iGPU and a dGPU running Ubuntu14.04? 15.8. What do I do if the display does not load, or CUDA does not work, after performing a system update? 15.9. How do I install a CUDA driver with a version less than 367 using a network repo? 15.10. How do I install an older CUDA version using a network repo? 15.11. Why does the installation on SUSE insta

In [ ]:
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_experimental.chat_models import Llama2Chat

In [ ]:
import json
from datetime import datetime
from typing import List, Dict
from transformers import AutoTokenizer, AutoModelForCausalLM

class CustomRetrieval:
    def retrieve(self, query: str) -> Dict:
        # Placeholder for your custom retrieval logic
        # This should return the results in the format you provided
        pass

class QueryEnhancer:
    def enhance(self, query: str) -> List[str]:
        # Placeholder for your query enhancement logic
        # This should return a list of expanded queries
        pass

class RAGModel:
    def __init__(self, model_name: str):
        self.retriever = CustomRetrieval()
        self.query_enhancer = QueryEnhancer()
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(model_name)

    def generate_answer(self, query: str) -> str:
        # Perform custom retrieval
        retrieval_results = self.retriever.retrieve(query)

        # Enhance the query
        expanded_queries = self.query_enhancer.enhance(query)

        # Combine original query, expanded queries, and retrieved documents
        context = f"Original query: {query}\n"
        context += f"Expanded queries: {', '.join(expanded_queries)}\n"
        context += "Retrieved documents:\n"
        for result in retrieval_results['results']:
            context += f"- {result['document']}\n"

        # Generate answer using the language model
        prompt = f"{context}\nBased on the above information, please answer the following question: {query}"
        input_ids = self.tokenizer.encode(prompt, return_tensors="pt")
        output = self.model.generate(input_ids, max_length=200, num_return_sequences=1, temperature=0.7)
        answer = self.tokenizer.decode(output[0], skip_special_tokens=True)

        return answer

    def process_query(self, query: str) -> Dict:
        timestamp = datetime.now().isoformat()
        retrieval_results = self.retriever.retrieve(query)
        expanded_queries = self.query_enhancer.enhance(query)
        answer = self.generate_answer(query)

        return {
            "timestamp": timestamp,
            "original_query": query,
            "expanded_queries": expanded_queries,
            "results": retrieval_results['results'],
            "answer": answer
        }

# Usage example
if __name__ == "__main__":
    model_name = "gpt2"  # Replace with your preferred model
    rag_model = RAGModel(model_name)
    
    query = "How do I install the Toolkit in a different location?"
    result = rag_model.process_query(query)
    print(json.dumps(result, indent=2))

In [1]:
from pymilvus import connections, Collection, utility

# Connect to Milvus
connections.connect("default", host="localhost", port="19530")

# List collections
collections = utility.list_collections()
print("Available collections:", collections)

Available collections: ['example_collection', 'documents']


In [2]:
collection_name = "documents"
collection = Collection(collection_name)

# Get collection information
print("Collection information:", collection.schema)

Collection information: {'auto_id': False, 'description': 'documents', 'fields': [{'name': 'id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': False}, {'name': 'embedding', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 768}}, {'name': 'content', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 65535}}], 'enable_dynamic_field': False}


In [3]:
results = collection.query(
    expr="id >= 0",  # Query condition, this will return all entities
    output_fields=["id"],  # Specify the fields you want to retrieve
    limit=10  # Limit the number of results
)

In [4]:
results

data: ["{'id': 1}", "{'id': 2}", "{'id': 3}", "{'id': 4}", "{'id': 5}", "{'id': 6}", "{'id': 7}", "{'id': 8}", "{'id': 9}", "{'id': 10}"] , extra_info: {'cost': 0}